In [1]:
import numpy as np
import pandas as pd

from IPython.display import SVG
from sknetwork.data import karate_club, painters, movie_actor
from sknetwork.clustering import Louvain, get_modularity
from sknetwork.linalg import normalize
from sknetwork.utils import get_membership
from sknetwork.visualization import svg_graph, svg_bigraph

from scipy import sparse

In [2]:
adj_mat = pd.read_csv('adj_mat.csv', sep=",", header=None).to_numpy()

In [3]:
adj_mat.shape

(5596, 5596)